<a href="https://colab.research.google.com/github/cool60334/111-TibaMe-AI-03-Machine-Learning/blob/main/ML_Naive_Bayes_poem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 先加入poem_test.csv，poem_train.csv兩個檔案

In [1]:
!pip install jieba

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd

In [3]:
import jieba

In [ ]:
traindf = pd.read_csv("poem_train.csv", encoding = "utf-8")
# traindf

In [ ]:
testdf = pd.read_csv("poem_test.csv", encoding = "utf-8")
# testdf

In [ ]:
def poem(s):
  s = s.replace("\r", "").replace("\n", "")  # 執行replace之後還是字串，可以再做replace
  return " ".join(jieba.cut(s))  # 用jieba.cut將字串分詞，並在字串間加入空白

traindf["內容"] = traindf["內容"].apply(poem)  # apply表示對每一格都做一次流程
# traindf

In [ ]:
testdf["內容"] = testdf["內容"].apply(poem)  # 對每一格都做一次流程
# testdf

In [19]:
# sklearn的答案一定要以0,1,2表示

trans = {"李白":0, "杜甫":1, "白居易":2}
traindf["作者"] = traindf["作者"].replace(trans)
# traindf

In [ ]:
testdf["作者"] = testdf["作者"].replace(trans)
# testdf

In [11]:
# 將文字特徵轉化為每個詞出現的次數

from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()

In [ ]:
trainvec = vec.fit_transform(traindf["內容"])
# print(trainvec)  # 印出訓練用資料內 (第n首詩, 第m個詞的欄位編號) 及 出現次數

In [ ]:
# print(vec.vocabulary_)  # 印出字典 {詞: 欄位編號}

In [ ]:
testvec = vec.transform(testdf["內容"])  # 測試用資料不需要fit
# print(testvec)  # 印出測試用資料內 (第n首詩, 第m個詞的欄位編號) 及 出現次數

In [15]:
# 訓練模型

from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()  # 特徵是整數
clf.fit(trainvec, traindf["作者"])  # 某作者寫的詩之中，出現某個詞的次數

MultinomialNB()

In [16]:
# 在預測用資料中，透過每首詩裡面的詞的出現次數，預測那首詩是哪位作者寫的

pre = clf.predict(testvec)
print("預測結果:", list(pre))
print("正確結果:", list(testdf["作者"]))

from sklearn.metrics import accuracy_score
print("正確率:", accuracy_score(pre, testdf["作者"]))

預測結果: [0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 1]
正確結果: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
正確率: 0.8


In [17]:
# 輸入一首詩來測試
s = input("請輸入一首詩: ")

# 當要預測時，要將資料轉成list或series

inputs = [poem(s)]  # 使用jieba分詞，並轉成list

x = vec.transform(inputs)  # 轉換成 (第n首詩, 第m個詞的欄位編號) 及 出現次數

rtrans = {0:"李白", 1:"杜甫", 2:"白居易"}
pre = clf.predict(x)[0]
print(f"這首詩是「{rtrans[pre]}」寫的")

請輸入一首詩: 天上何所有，迢迢白玉繩。 斜低建章闕，耿耿對金陵。 漢水舊如練，霜江夜清澄。 長川瀉落月，洲渚曉寒凝。 獨酌板橋浦，古人誰可徵。 玄暉難再得，灑酒氣填膺。
這首詩是「李白」寫的
